In [1]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
ex = "021"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [3]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':6,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 5000,
    'early_stopping_rounds': 200,
    'verbose_eval': 1000,
}

load_feature = ["../output/fe/fe001.feather",
                "../output/fe/fe002.feather",
                "../output/fe/fe003.feather",
                "../output/fe/fe004.feather",
                "../output/fe/fe005.feather",
                "../output/fe/fe006.feather",
                "../output/fe/fe007.feather",
                "../output/fe/fe009.feather",
                "../output/fe/fe011.feather",
                "../output/fe/fe012.feather",
                "../output/fe/fe020.feather",
                "../output/fe/fe021.feather",]

In [4]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [5]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-23 02:39:34,710 - INFO - logger set up


<RootLogger root (DEBUG)>

In [6]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        if "Source" in _df.columns:
            print(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

../output/fe/fe009.feather


In [7]:
for n,i in enumerate(train.columns):
    if i =="Source":
        print(n)

82


In [8]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0448026	valid_1's rmse: 0.317877
Early stopping, best iteration is:
[898]	training's rmse: 0.0504946	valid_1's rmse: 0.317656


2021-08-23 02:39:39,738 - INFO - Fold0:CV=0.3176559831488418
2021-08-23 02:39:39,743 - INFO - [fold 0] done in 5 s


Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0415255	valid_1's rmse: 0.310467
[2000]	training's rmse: 0.0202817	valid_1's rmse: 0.309279
[3000]	training's rmse: 0.0140165	valid_1's rmse: 0.308866
[4000]	training's rmse: 0.0103371	valid_1's rmse: 0.308587
[5000]	training's rmse: 0.00773145	valid_1's rmse: 0.308489
Did not meet early stopping. Best iteration is:
[5000]	training's rmse: 0.00773145	valid_1's rmse: 0.308489


2021-08-23 02:39:57,950 - INFO - Fold1:CV=0.3084889229817698
2021-08-23 02:39:57,958 - INFO - [fold 1] done in 18 s


Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0465619	valid_1's rmse: 0.349454
Early stopping, best iteration is:
[1290]	training's rmse: 0.0352454	valid_1's rmse: 0.349155


2021-08-23 02:40:05,698 - INFO - Fold2:CV=0.3491550164701295
2021-08-23 02:40:05,703 - INFO - [fold 2] done in 8 s


Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds


2021-08-23 02:40:09,917 - INFO - Fold3:CV=0.32944135182606643
2021-08-23 02:40:09,922 - INFO - [fold 3] done in 4 s


Early stopping, best iteration is:
[652]	training's rmse: 0.0736751	valid_1's rmse: 0.329441
Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[1000]	training's rmse: 0.0427317	valid_1's rmse: 0.340705
[2000]	training's rmse: 0.0210202	valid_1's rmse: 0.339783
Early stopping, best iteration is:
[1978]	training's rmse: 0.021249	valid_1's rmse: 0.339737


2021-08-23 02:40:22,585 - INFO - Fold4:CV=0.33973678106452915
2021-08-23 02:40:22,590 - INFO - [fold 4] done in 13 s
2021-08-23 02:40:22,601 - INFO - CV=0.32922109575786296
2021-08-23 02:40:22,609 - INFO - [lightgbm] done in 48 s
